# Variant Pathogenicity Prediction using DNABERT and Epigenomic Features

This project explores the use of pre-trained large language models (LLMs) for classifying variant pathogenicity, combining DNA sequence embeddings from DNABERT with epigenomic annotations from ENCODE.

## ✨ Highlights

- Uses **ClinVar** variant data with pathogenicity labels
- Integrates **DNase** and **H3K27ac** epigenomic tracks from ENCODE
- Generates **DNA sequence embeddings** with **DNABERT**
- Trains a supervised classifier to predict variant pathogenicity

In [1]:
#Install necessary packages
!pip install transformers datasets biopython scikit-learn --quiet pandas pybedtools biopython
!apt-get install -y tabix
!apt-get install -y bedtools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 24.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 31.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libhtscodecs2
The following NEW packages will be installed:
  libhtscodecs2 tabix
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 404 kB of archives.
After this operation, 1,300 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhtscodecs2 amd64 1.1.1-3 [53.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tabix amd64 1.13+ds-2build1 [351 kB]
Fetched 404 kB in 1s (352 kB/s)
S

In [2]:
#Import required libraries
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import pybedtools
import random
from pybedtools import BedTool
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
#Load a pretrained transformer model (we can replace this later with a domain-specific one)
model_name = "bert-base-uncased"  # We'll fine-tune later for DNA/epigenomic sequences
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
#simulate 1000 clinvar variants
random.seed(42)
def simulate_clinvar_variants(n=1000, benign_ratio=0.9):
    chromosomes = [f"chr{i}" for i in range(1, 23)] + ["chrX", "chrY"]
    bases = ["A", "T", "C", "G"]

    # Create label distribution
    num_benign = int(n * benign_ratio)
    num_pathogenic = n - num_benign
    labels = ["Benign"] * num_benign + ["Pathogenic"] * num_pathogenic
    random.shuffle(labels)

    data = {
        "chrom": [],
        "pos": [],
        "ref": [],
        "alt": [],
        "label": []
    }

    for label in labels:
        chrom = random.choice(chromosomes)
        pos = random.randint(10000, 1_000_000)
        ref = random.choice(bases)
        alt = random.choice([b for b in bases if b != ref])

        data["chrom"].append(chrom)
        data["pos"].append(pos)
        data["ref"].append(ref)
        data["alt"].append(alt)
        data["label"].append(label)

    return pd.DataFrame(data)

# Generate and save
clinvar_df = simulate_clinvar_variants()
clinvar_df.to_csv("clinvar_variants.tsv", sep="\t", index=False)
clinvar_df.head()

,chrom,pos,ref,alt,label
0,chr14,482387,G,T,Benign
1,chr2,252720,C,G,Benign
2,chr10,747135,G,A,Benign
3,chr22,254762,C,G,Pathogenic
4,chr19,703300,T,C,Benign


In [5]:
import pybedtools
from pybedtools import BedTool

# Convert DataFrame to BED format (start=pos-1, end=pos)
bed_lines = clinvar_df.apply(
    lambda row: f"{row.chrom}\t{row.pos-1}\t{row.pos}\t{row.ref}>{row.alt}\t{row.label}",
    axis=1
)
with open("clinvar.bed", "w") as f:
    f.write("\n".join(bed_lines))

clinvar = BedTool("clinvar.bed")
clinvar.head()

chr14	482386	482387	G>T	Benign
 chr2	252719	252720	C>G	Benign
 chr10	747134	747135	G>A	Benign
 chr22	254761	254762	C>G	Pathogenic
 chr19	703299	703300	T>C	Benign
 chr4	581014	581015	T>G	Benign
 chr5	962537	962538	C>A	Benign
 chr3	72534	72535	T>C	Benign
 chr20	795338	795339	C>T	Benign
 chr4	501464	501465	C>G	Benign
 

In [6]:
# Create a directory for ENCODE data
!mkdir -p encode_tracks
random.seed(42)
# Create a mock DNase-seq BED file
with open('encode_tracks/dnase_gm12878.bed', 'w') as f:
    for _ in range(1000):
        chrom = f"chr{random.randint(1, 22)}"
        start = random.randint(1000, 1_000_000)
        end = start + random.randint(100, 1000)
        f.write(f"{chrom}\t{start}\t{end}\n")

# Create a mock H3K27ac BED file
with open('encode_tracks/h3k27ac_gm12878.bed', 'w') as f:
    for _ in range(1000):
        chrom = f"chr{random.randint(1, 22)}"
        start = random.randint(1000, 1_000_000)
        end = start + random.randint(100, 1000)
        f.write(f"{chrom}\t{start}\t{end}\n")

In [7]:
# Load epigenomic tracks
dnase = BedTool('encode_tracks/dnase_gm12878.bed')
h3k27ac = BedTool('encode_tracks/h3k27ac_gm12878.bed')

In [8]:
!apt-get install -y bedtools
import pybedtools
pybedtools.helpers.set_tempdir('/tmp')  # Optional: define temporary directory

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bedtools is already the newest version (2.30.0+dfsg-2ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


Intersect ClinVar variants with epigenomic tracks:

In [9]:

# Intersect with DNase-seq peaks
overlap_dnase = clinvar.intersect(dnase, u=True)

# Intersect with H3K27ac peaks
overlap_h3k27ac = clinvar.intersect(h3k27ac, u=True)

print(len(overlap_h3k27ac))
print(len(overlap_dnase))
#for feature in overlap_h3k27ac:
    #print(feature)

23
27
chr5	146305	146306	C>A	Benign

chr13	31868	31869	C>A	Benign

chr19	684224	684225	C>G	Benign

chr3	611799	611800	T>G	Benign

chr12	590932	590933	C>T	Benign

chr21	715187	715188	G>T	Benign

chr10	749141	749142	T>G	Benign

chr21	990451	990452	G>T	Benign

chr2	989980	989981	A>C	Benign

chr5	813027	813028	G>T	Benign

chr12	997323	997324	G>C	Benign

chr5	893471	893472	T>A	Benign

chr5	291980	291981	C>T	Benign

chr5	329456	329457	A>C	Pathogenic

chr20	739065	739066	T>C	Benign

chr8	699544	699545	C>A	Benign

chr1	783206	783207	T>C	Benign

chr11	216646	216647	G>A	Benign

chr1	739385	739386	C>A	Benign

chr13	821031	821032	T>G	Benign

chr12	411582	411583	T>A	Benign

chr4	109612	109613	A>T	Benign

chr11	382389	382390	C>G	Benign



Annotate variants based on overlaps:

In [10]:
# Convert overlaps to sets for quick lookup
dnase_vars = set([line.name for line in overlap_dnase])
h3k27ac_vars = set([line.name for line in overlap_h3k27ac])

# Annotate the ClinVar DataFrame
import pandas as pd

clinvar_df = pd.read_csv('clinvar.bed', sep='\t', header=None, names=['chrom', 'start', 'end', 'name','label'])

def annotate(row):
    name = row['name']
    return pd.Series({
        'dnase_overlap': name in dnase_vars,
        'h3k27ac_overlap': name in h3k27ac_vars
    })

clinvar_df[['dnase_overlap', 'h3k27ac_overlap']] = clinvar_df.apply(annotate, axis=1)
clinvar_df.head()

,chrom,start,end,name,label,dnase_overlap,h3k27ac_overlap
0,chr14,482386,482387,G>T,Benign,True,True
1,chr2,252719,252720,C>G,Benign,False,True
2,chr10,747134,747135,G>A,Benign,True,True
3,chr22,254761,254762,C>G,Pathogenic,False,True
4,chr19,703299,703300,T>C,Benign,True,True


Add DNABERT Embeddings to the ML Pipeline

In [11]:
!pip install transformers
!pip install datasets

Load a Pretrained DNABERT Model.
We will use zhihan1996/DNA_bert_6 from Hugging Face.

In [12]:
from transformers import BertTokenizer, BertModel
import torch

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("zhihan1996/DNA_bert_6")
model = BertModel.from_pretrained("zhihan1996/DNA_bert_6")

model.eval()  # Set model to evaluation mode


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/359M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/359M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(4101, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

Prepare DNA Sequences Around Variants.
The variants are in clinvar_df with columns: chrom, start, end.

We need a reference genome (e.g., hg19/hg38 FASTA). For now, let’s simulate sequence fetching (we’ll later update it with actual genome access):

In [13]:
# function to simulate getting 101bp sequences around each variant
 # In real case: extract from reference genome using pyfaidx or samtools faidx
random.seed(42)
def get_mock_sequence(row):
    bases = ['A', 'C', 'G', 'T']
    sequence = ''.join(random.choices(bases, k=101))
    return sequence
clinvar_df['sequence'] = clinvar_df.apply(get_mock_sequence, axis=1)
clinvar_df.head()

,chrom,start,end,name,label,dnase_overlap,h3k27ac_overlap,sequence
0,chr14,482386,482387,G>T,Benign,True,True,GACAGGTACAAGAAGGAGTATGCATCAATGTGGTCGTGTGGAACAA...
1,chr2,252719,252720,C>G,Benign,False,True,GGGCGACCTTCGATTCGGATGTGACATTTCATTACATTACGCTCAG...
2,chr10,747134,747135,G>A,Benign,True,True,TACACACTCTCCTTGGACTGGGAGGTATAAGGAATAGGCGGTAGAC...
3,chr22,254761,254762,C>G,Pathogenic,False,True,GAACTTTTAAATTCGATTTTTAGCTTTTCTATTATCCTAAACTTCG...
4,chr19,703299,703300,T>C,Benign,True,True,TAAGGCTTGAAAACTACGAGCAGATTACATGAATCTGTGTTGGGTG...


In [14]:
#Tokenize and Embed the Sequences
def get_bert_embedding(seq):
    # Tokenize 6-mers
    kmer_seq = " ".join([seq[i:i+6] for i in range(len(seq) - 5)])
    inputs = tokenizer(kmer_seq, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze()  # Mean pool
    return embedding.numpy()

# Apply to your dataset (will take time – use tqdm for progress)
from tqdm import tqdm
sequence_embeddings = []

for seq in tqdm(clinvar_df['sequence']):
    embedding = get_bert_embedding(seq)
    sequence_embeddings.append(embedding)

# Convert list of arrays into DataFrame
import numpy as np
embedding_df = pd.DataFrame(sequence_embeddings)
embedding_df.columns = [f"embed_{i}" for i in range(embedding_df.shape[1])]
embedding_df.head()


100%|██████████| 1000/1000 [07:02<00:00,  2.37it/s]


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_758,embed_759,embed_760,embed_761,embed_762,embed_763,embed_764,embed_765,embed_766,embed_767
0,-0.199056,0.508052,-0.184158,-0.264671,0.322452,-0.378460,0.543806,-0.053282,-0.213419,0.591465,...,0.090525,-0.388747,0.210266,-0.154865,0.224566,0.662205,0.148829,-0.378555,-0.100452,0.552474
1,-0.414474,1.110924,-0.209952,-0.777064,0.305140,-0.369018,0.419810,-0.229718,-0.383574,0.220955,...,0.089958,-0.057903,0.307580,-0.482651,0.344147,0.984215,-0.217247,-0.704656,-0.046459,0.687657
2,-0.163201,0.559602,-0.293087,-0.363120,0.378611,-0.414554,0.379735,-0.159016,-0.231992,0.354319,...,-0.007785,-0.234342,0.297258,-0.238467,0.332171,0.859834,0.090143,-0.508501,-0.175291,0.362299
3,-0.305010,0.689446,-0.331987,-0.688610,0.088631,-0.385011,0.406878,0.012962,0.102733,-0.041133,...,0.176720,0.066540,0.262317,-0.251235,0.066995,0.845835,-0.184225,-0.586839,0.176243,0.421469
4,-0.318052,0.663465,-0.344164,-0.299088,0.216090,-0.503616,0.337484,-0.059180,-0.228937,0.295988,...,-0.136943,-0.049173,0.082337,-0.177305,0.114048,0.587887,0.121078,-0.389773,0.118951,0.387610


Combine Features and Train.
Merge DNABERT embeddings with the previous features (dnase_overlap, etc.)

In [15]:
full_df = pd.concat([clinvar_df[['label', 'dnase_overlap', 'h3k27ac_overlap']], embedding_df], axis=1)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X = full_df.drop(columns=['label'])
y = full_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

      Benign       0.93      0.99      0.96       186
  Pathogenic       0.00      0.00      0.00        14

    accuracy                           0.93       200
   macro avg       0.46      0.50      0.48       200
weighted avg       0.86      0.93      0.89       200

